In [152]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [153]:
df = pd.read_csv("imdb-movies-dataset.csv")
df.head()

,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...


In [154]:
df.drop(['Poster','Review Count','Review Title','Certificate','Metascore'],axis='columns',inplace=True)
df.head()

,Title,Year,Duration (min),Genre,Rating,Director,Cast,Votes,Description,Review
0,The Idea of You,2023.0,115.0,"Comedy, Drama, Romance",6.4,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...","This film, as well as the reaction to it, is a..."
1,Kingdom of the Planet of the Apes,2023.0,145.0,"Action, Adventure, Sci-Fi",7.3,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...","I'm a big fan of all the planet of the apes, a..."
2,Unfrosted,2023.0,97.0,"Biography, Comedy, History",5.5,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",Pretty much the worst criticism you can lay on...
3,The Fall Guy,2023.0,126.0,"Action, Comedy, Drama",7.3,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,Just got out of the Austin premier at SXSW and...
4,Challengers,2023.0,131.0,"Drama, Romance, Sport",7.7,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",This is a tough one. I liked the concept and t...


In [155]:
df.isna().sum()

Title               0
Year              150
Duration (min)    336
Genre               7
Rating            404
Director            5
Cast               39
Votes             404
Description         0
Review            516
dtype: int64

In [156]:
df[['Review','Cast','Director','Genre']] = df[['Review','Cast','Director','Genre']].fillna("Not available")
df[['Year','Duration (min)','Rating','Votes']] = df[['Year','Duration (min)','Rating','Votes']].fillna("Not available")

In [157]:
df.isna().sum()

Title             0
Year              0
Duration (min)    0
Genre             0
Rating            0
Director          0
Cast              0
Votes             0
Description       0
Review            0
dtype: int64

In [158]:
df['combined_text'] =  df['Year'].astype(str) + " " + df['Genre'] + " " + df['Director'] + " " + df['Cast']
df.head()

,Title,Year,Duration (min),Genre,Rating,Director,Cast,Votes,Description,Review,combined_text
0,The Idea of You,2023.0,115.0,"Comedy, Drama, Romance",6.4,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...","This film, as well as the reaction to it, is a...","2023.0 Comedy, Drama, Romance Michael Showalte..."
1,Kingdom of the Planet of the Apes,2023.0,145.0,"Action, Adventure, Sci-Fi",7.3,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...","I'm a big fan of all the planet of the apes, a...","2023.0 Action, Adventure, Sci-Fi Wes Ball Owen..."
2,Unfrosted,2023.0,97.0,"Biography, Comedy, History",5.5,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",Pretty much the worst criticism you can lay on...,"2023.0 Biography, Comedy, History Jerry Seinfe..."
3,The Fall Guy,2023.0,126.0,"Action, Comedy, Drama",7.3,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,Just got out of the Austin premier at SXSW and...,"2023.0 Action, Comedy, Drama David Leitch Ryan..."
4,Challengers,2023.0,131.0,"Drama, Romance, Sport",7.7,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",This is a tough one. I liked the concept and t...,"2023.0 Drama, Romance, Sport Luca Guadagnino Z..."


In [159]:
df['combined_text'].iloc[0]


'2023.0 Comedy, Drama, Romance Michael Showalter Anne Hathaway, Nicholas Galitzine, Ella Rubin, Annie Mumolo'

In [160]:
df['Votes'].apply(type).value_counts()

Votes
<class 'str'>    10000
Name: count, dtype: int64

In [161]:
tfidf = TfidfVectorizer()
combined_vector = tfidf.fit_transform(df['combined_text'])

In [162]:
match = "tom cruise"
match_tfidf = tfidf.transform([match])
match

'tom cruise'

In [163]:
similarities = cosine_similarity(match_tfidf, combined_vector)
top_indices = similarities[0].argsort()[::-1][:5]

In [166]:
df.iloc[top_indices][['Title','Director','Rating']]

,Title,Director,Rating
9535,Untitled Tom Cruise/SpaceX Project,Doug Liman,Not available
6475,Top Gun 3,Glen Powell,Not available
3884,Days of Thunder,Tony Scott,6.1
4844,Far and Away,Ron Howard,6.6
699,A Few Good Men,Rob Reiner,7.7
